In [5]:
import numpy as np
import pandas as pd
import math

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [56]:
columns = ['Date', 'Time', 'Choke', 'Psig', 'APsig', 'BOPD (Oil)', 'MMSCFD (Gas)', 'BWPD (Air)']

In [57]:
df_data = pd.read_excel('data.xlsx')
df_data = df_data[columns]

In [58]:
# fill error data
df_data = df_data.ffill()

In [59]:
datetime = pd.to_datetime(df_data['Date'].map(str) + ' ' + df_data['Time'].map(str))

/tmp/ipykernel_31566/3734410702.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  datetime = pd.to_datetime(df_data['Date'].map(str) + ' ' + df_data['Time'].map(str))


In [60]:
df_data['Datetime'] = datetime
df_data = df_data.set_index('Datetime')

In [61]:
df_data = df_data.drop('Date', axis=1)
df_data = df_data.drop('Time', axis=1)

In [62]:
df_data

,Choke,Psig,APsig,BOPD (Oil),MMSCFD (Gas),BWPD (Air)
Datetime,,,,,,
2021-05-15 21:00:00,38,935,320,698.900,4.980,23.600
2021-05-15 21:15:00,38,935,321,822.500,4.988,32.600
2021-05-15 21:30:00,38,940,318,661.400,5.070,26.200
2021-05-15 21:45:00,38,938,317,737.700,5.061,29.200
2021-05-15 22:00:00,38,940,319,661.400,5.078,26.200
...,...,...,...,...,...,...
2021-01-06 11:00:00,42,988,327,817.992,7.534,17.856
2021-01-06 11:15:00,42,990,328,809.208,7.545,17.664
2021-01-06 11:30:00,42,986,326,810.048,7.522,17.664


In [10]:
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(df_data)

TypeError: float() argument must be a string or a real number, not 'datetime.datetime'